# Train a FastText model on the SLED categorization dataset

In [7]:
# Install FastText
%git clone https://github.com/facebookresearch/fastText.git
%cd fastText
python setup.py install

ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
You should consider upgrading via the 'c:\users\tajak\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [14]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip

%cd fastText-0.9.2

!pip install .

import fasttext as ft

'wget' is not recognized as an internal or external command,
operable program or batch file.


[WinError 2] The system cannot find the file specified: 'fastText-0.9.2'
c:\Users\tajak\Google Drive\GitHub\FastText-Classification-on-SLED


'unzip' is not recognized as an internal or external command,
operable program or batch file.
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
You should consider upgrading via the 'c:\users\tajak\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'fasttext'

In [16]:
%cd fastText-0.9.2
%pwd

[WinError 2] The system cannot find the file specified: 'fastText-0.9.2'
c:\Users\tajak\Google Drive\GitHub\FastText-Classification-on-SLED\fastText-0.9.2


'c:\\Users\\tajak\\Google Drive\\GitHub\\FastText-Classification-on-SLED\\fastText-0.9.2'

In [18]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating python\fasttext_module\fasttext.egg-info
writing python/fasttext_module\fasttext.egg-info\PKG-INFO
writing dependency_links to python/fasttext_module\fasttext.egg-info\dependency_links.txt
writing requirements to python/fasttext_module\fasttext.egg-info\requires.txt
writing top-level names to python/fasttext_module\fasttext.egg-info\top_level.txt
writing manifest file 'python/fasttext_module\fasttext.egg-info\SOURCES.txt'
adding license file 'LICENSE' (matched pattern 'LICEN[CS]E*')
reading manifest file 'python/fasttext_module\fasttext.egg-info\SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'python/fasttext_module\fasttext.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
creating build
creating build\lib.win-amd64-3.9
creating build\lib.win-amd64-3.9\fasttext
copying python\fasttext_module\fasttext\FastText.py -> build\lib.win-amd64-3.9

c:\Users\tajak\AppData\Local\Programs\Python\Python39\lib\site-packages\setuptools\dist.py:642: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  warnings.warn(
error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/


In [20]:
import fastText as ft

In [30]:
!pip install parse

  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24581 sha256=4a8d20c6e1e5f1885710520ff470cd14e6f95db73145e1cda465d5f24b299037
  Stored in directory: c:\users\tajak\appdata\local\pip\cache\wheels\d6\9c\58\ee3ba36897e890f3ad81e9b730791a153fce20caa4a8a474df
Successfully built parse


You should consider upgrading via the 'c:\users\tajak\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
# Import necessary libraries
import fastText as ft
import parse
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [2]:
# Define paths to data
train = "data/sled_categorization_TRAINSMALL_PREPROCESSED.txt"

test = "data/sled_categorization_TEST_PREPROCESSED.txt"

dev = "data/sled_categorization_DEV_PREPROCESSED.txt"

In [3]:
# Parse the file to separate labels from text
def parse_file(path: str):
    """Reads fasttext formatted file and returns labels, texts."""
    with open(path, "r") as f:
        content = f.readlines()
    pattern = "{label} {text}\n"
    p = parse.compile(pattern)

    labels, texts = list(), list()
    for line in content:
        rez = p.parse(line)
        if rez is not None:
            labels.append(rez["label"])
            texts.append(rez["text"])
        else:
            print("error parsing line ", line)
    return labels, texts

In [4]:
# Create a list of texts and labels from the train file
train_labels, train_texts = parse_file(train)

# Inspect the results
train_texts[:3]

['na tolminskem se je dopoldne zgodila prometna nesreča v kateri je umrl 43-letni voznik osebnega avtomobila po tem ko je zapeljal v 100 metrov globoko grapo  nesreča se je zgodila ob 916 uri po prvih ugotovitvah policistov je domačin peljal iz tolmina proti vasi zadlaz - žabče kjer je v neposredni bližini ene od tamkajšnjih stanovanjskih hiš zapeljal levo z vozišča globoko približno 100 metrov v grapo  poleg policistov iz tolmina in nove gorice ter novogoriške gorske policijske enote so na kraju nesreče posredovali tudi gasilci pripadniki gorske reševalne službe in reševalci iz tolmina v nadaljevanju je bila ugotovljena in potrjena identiteta voznika in sicer je za posledicami telesnih poškodb v prometni nesreči podlegel 43-letni voznik so sporočili iz policijske uprave nova gorica zdravnica je odredila sanitarno obdukcijo ki jo bodo opravili na inštitutu za sodno medicino v ljubljani o nesreči so obvestili tudi preiskovalno sodnico in državno tožilstvo v novi gorici',
 'na cesti zadl

In [5]:
# Create a list of labels - by creating a set, we get only unique labels, then we transform the set back to a list
LABELS = list(set(train_labels))

LABELS

['__label__okolje',
 '__label__zdravje',
 '__label__gospodarstvo',
 '__label__kultura',
 '__label__crnakronika',
 '__label__vreme',
 '__label__zabava',
 '__label__izobrazevanje',
 '__label__prosticas',
 '__label__druzba',
 '__label__sport',
 '__label__znanost',
 '__label__politika']

## Hyperparameters search

In [6]:
def prediction_to_label(prediction):
    """Transforms predictions as returned by fasttext into pure labels."""
    return np.array(prediction[0])[:, 0]

# Create a function to evaluate the results (it calculates the scores and creates a confusion matrix)
def plot_cm():
    """
    Plots confusion matrix for prediction on the test set.
    Takes the predictions, named as y_pred, true values, named as y_true,
    and labels, named as LABELS.
    
    Arguments:
        save: whether the confusion matrix is saved. Defaults to False.
        title: the title of the confusion matrix. Defaults to None.
    """
    cm = confusion_matrix(y_true, y_pred, labels=LABELS)
    plt.figure(figsize=(9, 9))
    plt.imshow(cm, cmap="Oranges")
    classNames = LABELS
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(classNames))
    plt.xticks(tick_marks, classNames, rotation=90)
    plt.yticks(tick_marks, classNames)
    microF1 = f1_score(y_true, y_pred, labels=LABELS, average ="micro")
    macroF1 = f1_score(y_true, y_pred, labels=LABELS, average ="macro")

    print(f"{microF1:0.4}")
    print(f"{macroF1:0.4}")

    metrics = f"{microF1:0.4}, {macroF1:0.4}"
    plt.tight_layout()
    plt.show()
    return microF1, macroF1

In [7]:
# Create a list with results
results = []

### Hyperparameter search - epoch number

I'll experiment with various epochs to see which epoch number provides the best results

In [30]:
# Define which epochs we want to try
exp_range = [10, 50, 100, 150]

for i in exp_range:
    print(f"Training started for epoch_num {i}.")

    model = ft.train_supervised(input=train,
                                epoch = i,
                                #lr = 0.7,
                                #wordNgrams=1,
                                verbose = 2
                                            )

    print(f"Training finished. Testing started.")

    # Parse the dev files so that labels and texts are separated
    y_true, y_texts = parse_file(dev)

    # Evaluate the model on dev data
    y_pred = model.predict(y_texts)
    y_pred = prediction_to_label(y_pred)

    # Plot the confusion matrix:
    m, M = plot_cm()
    
    rezdict = dict(
        microF1=m,
        macroF1=M,
        epoch_num = epoch,
    )
    results.append(rezdict)

Training started for epoch_num 10.


AttributeError: module 'fastText' has no attribute 'train_supervised'

In [ ]:
# Test the model
model.predict("V Celju se je zgodila nesreča, umrla je 42-letna ženska.")

In [ ]:
# Analyse the results of experiments
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.style.use('_mpl-gallery')

# make data

x = []
mi = []
ma = []

for i in results:
    x.append(i['ngram'])
    mi.append(i['microF1'])
    ma.append(i["macroF1"])

# plot
fig, ax = plt.subplots(figsize=(6,3), dpi=100)

ax.plot(x, mi, linewidth=2.0, label="Micro F1")
ax.plot(x, ma, linewidth=2.0, label="Macro F1")

ax.set(xlim=(min(exp_range), max(exp_range)),
       ylim=(0.5, 0.7),xticks=exp_range)

ax.set_xlabel('Word n-Grams')
ax.set_ylabel('F1 Score')
ax.legend(loc="lower right")

plt.tight_layout()
plt.savefig("N-grams.png")
plt.show()

In [ ]:
# Save results as JSON file
import json

with open("fastText_results.json","w") as file:
  json.dump(results, file)

## Using the Slovene embeddings

In [8]:
# Unzip a file
# importing necessary modules
import wget, zipfile
#from io import BytesIO
print('Downloading started')

#Defining the zip file URL
url = "https://www.clarin.si/repository/xmlui/bitstream/handle/11356/1204/embed.sl-token.ft.sg.vec.zip"

# Split URL to get the file name
filename = url.split('/')[-1]

# Downloading the file by sending the request to the URL
embs = wget.download(url_name)
print('Downloading Completed')

# extracting the zip file contents
zipfile= zipfile.ZipFile(embs)
zipfile.extractall('data/')

In [ ]:
exp_range = [1,2,3,4,5]

for i in exp_range:
    model = ft.train_supervised(input=train,
                                epoch = 350,
                                lr = 0.7,
                                wordNgrams=1,
                                verbose = 2,
                                pretrainedVectors = "data/embed.sl-token.ft.sg.vec"
                                            )
    # Parse the dev files so that labels and texts are separated
    y_true, y_texts = parse_file(dev)

    # Evaluate the model on test data
    y_pred = model.predict(y_texts)
    y_pred = prediction_to_label(y_pred)

    # Plot the confusion matrix:
    m, M = plot_cm(save=False, title=f"NGram: {i}")
    
    rezdict = dict(
        microF1=m,
        macroF1=M,
        ngram=i
    )
    results.append(rezdict)